In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from captum.attr import IntegratedGradients
import os


/var/folders/lv/hk3y0lgs0nngk_1q5qq128rc0000gn/T/ipykernel_77249/1875987159.py:45: DtypeWarning: Columns (22,23,24,26,27,28,29,30,31,36,41,43,58,59,60,61) have mixed types. Specify dtype option on import or set low_memory=False.
  df_X = pd.read_csv(INPUT_CSV)


ValueError: could not convert string to float: 'PAOOWF3GUFM46FBSP561'

In [ ]:

# TODO: save cleaned CSV used for training and use same here

MODEL_PATH = "../training/ff_model.pth"  # path to saved state_dict
INPUT_CSV = "../datasets/2024_lar_cleaned.csv"       # CSV to explain
OUT_DIR = "ig_outputs"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BASELINE = "zeros"
N_STEPS = 50

os.makedirs(OUT_DIR, exist_ok=True)


In [ ]:
class FeedForwardNN(nn.Module):
    def __init__(self, input_dim, hidden_dims=[256,128,64], num_classes=8, dropout=0.5):
        super().__init__()
        layers = []
        prev_dim = input_dim
        for h in hidden_dims:
            layers.append(nn.Linear(prev_dim, h))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            prev_dim = h
        layers.append(nn.Linear(prev_dim, num_classes))
        self.network = nn.Sequential(*layers)
    def forward(self, x):
        return self.network(x)


In [ ]:

df_X = pd.read_csv(INPUT_CSV)
features = df_X.columns.tolist()
X = df_X.values.astype(np.float32)


input_dim = X.shape[1]
num_classes = 8  # adjust to your dataset
model = FeedForwardNN(input_dim=input_dim, hidden_dims=[256,128,64], num_classes=num_classes, dropout=0.5)
state_dict = torch.load(MODEL_PATH, map_location=DEVICE)
model.load_state_dict(state_dict)
model.to(DEVICE)
model.eval()



def forward_func(x):
    out = model(x)
    if out.shape[1] > 1:
        probs = torch.softmax(out, dim=1)
        return probs[:,1]   # probability of class 1
    else:
        return torch.sigmoid(out.squeeze(1))

ig = IntegratedGradients(forward_func)

def to_tensor(x_row):
    return torch.tensor(x_row, dtype=torch.float32, device=DEVICE, requires_grad=True)



abs_accum = np.zeros(X.shape[1])
for i, row in enumerate(X):
    x_tensor = to_tensor(row).unsqueeze(0)
    baseline = torch.zeros_like(x_tensor) if BASELINE=="zeros" else torch.tensor(X.mean(axis=0), device=DEVICE).unsqueeze(0)
    attr, delta = ig.attribute(x_tensor, baselines=baseline, n_steps=N_STEPS, return_convergence_delta=True)
    attr = attr.detach().cpu().numpy().flatten()
    abs_accum += np.abs(attr)

    df_attr = pd.DataFrame({"feature": features, "attribution": attr, "abs_attribution": np.abs(attr)})
    df_attr = df_attr.sort_values("abs_attribution", ascending=False)
    df_attr.to_csv(os.path.join(OUT_DIR, f"ig_sample_{i}.csv"), index=False)
    print(f"Saved IG for sample {i}, delta={float(delta):.4e}")



mean_abs = abs_accum / X.shape[0]
agg_df = pd.DataFrame({"feature": features, "mean_abs_attribution": mean_abs}).sort_values("mean_abs_attribution", ascending=False)
agg_df.to_csv(os.path.join(OUT_DIR, "ig_mean_abs_attributions.csv"), index=False)

print("\nTop features for sample 0:")
print(pd.read_csv(os.path.join(OUT_DIR, "ig_sample_0.csv")).head(10).to_string(index=False))

print("\nTop global features (mean abs):")
print(agg_df.head(10).to_string(index=False))
